# Joint Intent detection and slot filling
Dieses Jupyter notebook wurde als semesterabschließende Arbeit für das Modul Natural Language Processing an der [Fachhochschule Südwestfalen](https://www.fh-swf.de/en/international_3/index.php) erstellt.

## Einleitung
Das Joint intent detection and slot filling (IDSF) ist eine Aufgabe aus dem Teilbereich des Natural Language Understandings (NLU) des Natural Language Processings (NLP), die uns in heutzutage fast täglich Alltag begegnet. Sei es um einen Timer auf dem Handy zu starten, bestimmte Musik abzuspielen oder das Licht einzuschalten. Der Ablauf ist dabei häufig der selbe: "Siri stelle einen Timer für 4 Minuten", "Alexa spiele meine Schlager Playlist" oder "Google erstelle einen Arzttermin für heute 16:00 Uhr". Meist beginnen die Kommandos mit dem Namen des Sprachassistenten, um diesen zu aktivieren, gefolgt vom Kommando für die gewünschte Aktion. Das IDSF beschäftigt sich dabei mit der Aufgabe, die gewünschte Aktion (Intent), also stelle einen Timer, Spiele Musik, erstelle einen Termin im Kalender und die dazugehörigen notwendigen Parameter, wie z.B. vier Minuten, Schlager Playlist oder Arzt heute 16:00 Uhr (Slots) zu erkennen.
Da der Gebrauch dieser Sprachassistenten in Zukunft wahrscheinlich noch stärker zu nehmen wird, wollen wir uns deren funktionsweise in diesem Notebook näher anschauen. Dafür wird zuerst die Entwicklungshistorie vom IDSF betrachtet und anschließend wird ein eigenes Modell für die Erkennung erstellt und anhand eines selbst vorbereiteten Korpus trainiert.

## Joint Intent Detection and Slot filling

--------------

## Datenbeschaffung

Als Datensatz für das nachfolgende Beispiel verwenden wir den Snips-Datensatz. Dieser Datensatz wurde vom, mittlerweil zu Sonos gehörenden [1], [Snips Team](https://snips.ai/) zusammengestellt, um ihr eigenes Modell mit anderen Wettbewerbern wie zum Beispiel Amazons Alexa zu verglichen. Die Ergebnisse und die Datensätze der drei Vergleiche wurden in einem [GitHub Repository](https://github.com/sonos/nlu-benchmark/tree/master) veröffentlicht und in dem Paper "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces" [2] erläutert. Das Repository enthält die Daten für drei Evaluationen aus den Jahren 2016 bis 2018. Wir werden in diesem Notebook die Daten der 2017 durchgeführten Evaluation verwenden, da diese Sätze für sieben unterschiedliche und allgemeine Aufgaben enthält.

Die Daten sind im dem Repository in einzelnen JSON-Dateien enthalten. Dabei gibt es pro Aufgabe zwei Dateien, eine für das Training und eine für die Validierung. Der Einfachheit halber wurden die Dateien in dem data Verzeichnis, dass diesem Notebook beiliegt, abgelegt.

Nachfolgend ist ein Auszug aus der `train_AddToPlaylist_full.json`-Datei.

In [15]:
!head -n 48 data/train_AddToPlaylist_full.json # Zeige die ersten 23 Zeilen der angegebenen Datei an.

{
  "AddToPlaylist": [
    {
      "data": [
        {
          "text": "Add another "
        },
        {
          "text": "song",
          "entity": "music_item"
        },
        {
          "text": " to the "
        },
        {
          "text": "Cita Romántica",
          "entity": "playlist"
        },
        {
          "text": " playlist. "
        }
      ]
    },
    {
      "data": [
        {
          "text": "add "
        },
        {
          "text": "clem burke",
          "entity": "artist"
        },
        {
          "text": " in "
        },
        {
          "text": "my",
          "entity": "playlist_owner"
        },
        {
          "text": " playlist "
        },
        {
          "text": "Pre-Party R&B Jams",
          "entity": "playlist"
        }
      ]
    },


Die Datei besteht an oberster Stelle aus dem Namen der gewünschten Aktion gefolgt von einer Liste an Objekten mit einem `Data` Attribut. Dieses enthält wiederum eine Liste von Objekten mit `Text` Attributen die den Satz in Teilen enthält. Dabei wird der Satz durch den Text eines definierten `Entities` geteilt. So enthält das erste Beispiel den Text bis zum ersten `entity` das als `music_item` klassifiziert wurde und wieder den gesamten Text bis zum nächsten entity, dem Namen einer Playlist.

Als nächstes werden die Daten in ein verwendbares Format transformiert. Ein in der Natural language processing gängiges Format ist das IOB Format. IOB steht für Inside-Outside-Beginning. Dieses Format ermöglicht die Kennzeichnung der einzelnen Entitäten in einem Satz. Es wird unteranderem von `NLTK` und `spaCy` unterstützt [3, 4].


Hier ein Beispiel für das Zielformat:

    add clem burke in my playlist Pre-Party R&B Jams    AddToPlaylist o o b-mucic_item i-music_item o b-playlist_owner o b-playlist i-playlist i-playlist
    
Am Beginn der Zeile steht der vollständige Satz. Dahinter wird die Aktion (Intent) durch einen Tab getrennt definiert. Nun folgengen für jeden Token entweder der Buchstabe 'o', dieser steht für keine Bedeutung, der Buchstabe 'b', für den Beginn einer Entität, oder 'i', als Entität die zugehörig zum voherigen 'b' ist. Das 'b' und 'i' werden dabei jeweils gefolgt vom einem trennenden Bindestrich und der Entität. So ist der Name 'Clem Burke' unterteilt in ein `b-music_item` für Clem und `i-Music_item` für Burke. Dadurch wird definiert, dass die beiden Teile zusammen gehören.

Quellen

* 1: https://investors.sonos.com/news-and-events/investor-news/latest-news/2019/Sonos-Announces-Acquisition-of-Snips/default.aspx, [Online, 07.03.2025]
* 2: Coucke A. et al., "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces." 2018, [Online: https://arxiv.org/abs/1805.10190, 07.03.2025]
* 3: NLTK Team, tree2conlltags, [Online: https://www.nltk.org/_modules/nltk/chunk/util.html#tree2conlltags, 13.03.2025]
* 4: Explosion, spaCy convert, [Online: https://spacy.io/api/cli#converters, 13.03.2025]

## Erstellen eines Corpus